In [12]:
import urllib3
from bs4 import BeautifulSoup
import requests
import certifi
import time
import datetime

In [94]:
#!/usr/bin/python
import MySQLdb

db = MySQLdb.connect(host="localhost",    # your host, usually localhost
                     user="root",         # your username
                     passwd="root",  # your password
                     db="pipa_db")        # name of the data base

cur = db.cursor()

In [95]:
show_tables = "show tables"
cur.execute(show_tables)

6L

In [96]:
dailyMax = 1000
batchNum = 20
sleeptimeforPubmed =5
backstep =20

cur.execute('select pmid from last_pmid')
endingPMIDtuple = max(cur.fetchall())
endingPMID = endingPMIDtuple[0]
startingPMID=28314358 #endingPMID -backstep
print endingPMID, startingPMID
    

28314369 28314358


In [90]:
cur.execute('truncate articles')
# erase data or re-create articles table

0L

In [91]:
%%time
        for i in range (startingPMID, startingPMID+dailyMax, batchNum):
            PMIDs = ""

            for j in range (batchNum):
                PMIDs = PMIDs + str(i + j ) +","

            url = "https://www.ncbi.nlm.nih.gov/pubmed/"+ PMIDs +"?report=medline&format=text" 

            #time.sleep(sleeptimeforPubmed)
            http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED',ca_certs=certifi.where())
            r = http.request('GET', url)
            soup = BeautifulSoup(r.data, "lxml")

            allInfo = unicode(soup.pre.string)

            if allInfo.find('PMID- ')<0:
                print url
                print 'bad end'        
                break

            while allInfo.find('AB  -')>0:
                IdIndexStart = allInfo.index('PMID- ')+6
                IdIndexEnd = allInfo.index('\n', IdIndexStart)
                PMID = allInfo[IdIndexStart:IdIndexEnd]

                tiIndexStart = allInfo.index('TI  -')+6
                tiIndexEnd =   allInfo.index('PG  -', tiIndexStart)-1
                ti = allInfo[tiIndexStart:tiIndexEnd]
                
                abIndexStart = allInfo.index('AB  -')+6
                abIndexEnd =   allInfo.index('FAU -', abIndexStart)-1
                ab = ti+"        Abstract:"+allInfo[abIndexStart:abIndexEnd]

                query = "insert into articles (pmid, abstract) values ("+ PMID +", '"+ ab + "')"
                endingPMID = int(PMID)
                if (endingPMID%10==0):
                    print endingPMID

                try:
                    cur.execute(query)
                    db.commit()

                except:
                    db.rollback()

                allInfo = allInfo[abIndexEnd:]

        now = datetime.datetime.now()       
        query = 'insert into last_pmid (record_date, pmid) values ("%s", %d)'%(now, endingPMID)
        cur.execute(query)
        db.commit()
# write endingPMID to db
db.close()    
    

28314360
https://www.ncbi.nlm.nih.gov/pubmed/28314378,28314379,28314380,28314381,28314382,28314383,28314384,28314385,28314386,28314387,28314388,28314389,28314390,28314391,28314392,28314393,28314394,28314395,28314396,28314397,?report=medline&format=text
bad end
CPU times: user 41.8 ms, sys: 3.19 ms, total: 45 ms
Wall time: 991 ms
